In [1]:
from datetime import datetime
import datetime as dt
import xarray as xr
import numpy as np
import glob
from dask.distributed import Client

from datacube.api import GridWorkflow

from madmex.wrappers import gwf_query
from madmex.util.xarray import to_float, to_int


2019-12-03 19:35:18,419 - datacube.drivers.driver_cache - driver_cache safe_load: Failed to resolve driver datacube.plugins.index::s3aio_index
2019-12-03 19:35:18,421 - datacube.drivers.driver_cache - driver_cache safe_load: Error was: UndefinedEnvironmentName("'extra' does not exist in evaluation environment.",)
  Anticipated allocation: 1368281KB
2019-12-03 22:11:39,059 - madmex.wrappers - loggerwriter write:   Anticipated allocation: 1368281KB
2019-12-03 22:11:39,058 - bis - segment segmentor:   Anticipated allocation: 1368281KB
    ...allocated
2019-12-03 22:11:39,062 - madmex.wrappers - loggerwriter write:     ...allocated
2019-12-03 22:11:39,062 - bis - segment segmentor:     ...allocated
  thresh 1
2019-12-03 22:12:48,450 - madmex.wrappers - loggerwriter write:   thresh 1
2019-12-03 22:12:48,450 - bis - segment segment_gen:   thresh 1
  thresh 2
2019-12-03 22:15:55,725 - madmex.wrappers - loggerwriter write:   thresh 2
2019-12-03 22:15:55,725 - bis - segment segment_gen:   thres

In [2]:
scheduler_file = '/shared_volume/scheduler.json'

In [3]:
client = Client(scheduler_file=scheduler_file)

In [4]:
client

Client Scheduler: tcp://10.0.0.3:8786 Dashboard: http://10.0.0.3:8787/status,Cluster Workers: 5 Cores: 5 Memory: 500.00 GB


In [5]:
client.restart()

Client Scheduler: tcp://10.0.0.3:8786 Dashboard: http://10.0.0.3:8787/status,Cluster Workers: 5 Cores: 5 Memory: 500.00 GB


In [6]:
region = 'Veracruz'
name_of_product = 's2_l2a_10m_scl_mexico'
begin = '2018-01-01'
end = '2019-12-31'
gwf_kwargs = {'region': region, 'product': name_of_product, 'begin': begin, 'end':end}
iterable = gwf_query(**gwf_kwargs)

In [7]:
lista_iter = list(iterable)

In [8]:
dc_tile=(39,-30)
index = [x for x in range(0,len(lista_iter)) if lista_iter[x][0] == dc_tile]

In [9]:
[lista_iter[x][0] for x in range(0,len(lista_iter))]

[(40, -33),
 (41, -33),
 (42, -33),
 (40, -32),
 (42, -32),
 (41, -32),
 (40, -31),
 (41, -31),
 (42, -31),
 (40, -30),
 (40, -29),
 (41, -30),
 (39, -31),
 (38, -30),
 (39, -30),
 (38, -29),
 (39, -29),
 (39, -32)]

In [10]:
var = lista_iter[index[0]]

In [11]:
var

((39, -30), Tile<sources=<xarray.DataArray (time: 63)>
 array([(Dataset <id=828bc7bd-5117-4255-97cc-ff518ca81f6f type=s2_l2a_10m_scl_mexico location=file:///shared_volume/datacube/datacube_ingest/s2_l2a_10m_scl_s3/mexico/s2_l2a_10m_scl_s3_39_-30_20181004170824000000.nc>,),
        (Dataset <id=be1f83ae-43b2-4e98-b0d2-92634244e9f3 type=s2_l2a_10m_scl_mexico location=file:///shared_volume/datacube/datacube_ingest/s2_l2a_10m_scl_s3/mexico/s2_l2a_10m_scl_s3_39_-30_20181006170522000000.nc>,),
        (Dataset <id=7b722688-3566-4208-808b-a6bdcd47c56a type=s2_l2a_10m_scl_mexico location=file:///shared_volume/datacube/datacube_ingest/s2_l2a_10m_scl_s3/mexico/s2_l2a_10m_scl_s3_39_-30_20181009171234000000.nc>,),
        (Dataset <id=03f1eeeb-65a4-4aad-a130-10ca6bdcfbf8 type=s2_l2a_10m_scl_mexico location=file:///shared_volume/datacube/datacube_ingest/s2_l2a_10m_scl_s3/mexico/s2_l2a_10m_scl_s3_39_-30_20181031170221000000.nc>,),
        (Dataset <id=03f1c839-d9c9-4322-a17d-72135c3e2fb6 type=s2_l2a

In [12]:
sr_0 = GridWorkflow.load(var[1],dask_chunks={'x': 800, 'y': 800})


In [13]:
sr_0

<xarray.Dataset>
Dimensions:   (time: 63, x: 5002, y: 5002)
Coordinates:
  * time      (time) datetime64[ns] 2018-10-04T17:08:24 ... 2019-09-26T17:08:12.480000
  * y         (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x         (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06
Data variables:
    blue      (time, y, x) uint16 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    green     (time, y, x) uint16 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    red       (time, y, x) uint16 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    nir       (time, y, x) uint16 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    pixel_qa  (time, y, x) uint16 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...

In [14]:
crs = var[1].geobox.crs

In [15]:
sr_0.attrs['geobox'] = var[1].geobox

In [16]:
sr_0 = sr_0.apply(func=to_float, keep_attrs=True)

In [17]:
sr_0 = client.persist(sr_0)

In [18]:
sr_1 = sr_0.where(sr_0.pixel_qa.isin([2,4,5,6,7,8,11]))

In [19]:
sr_1['ndvi'] = ((sr_1.nir - sr_1.red) / (sr_1.nir + sr_1.red)) * 10000

In [20]:
sr_1['ndvi'].attrs['nodata'] = 0

In [21]:
#sr_1 = sr_1.drop(['red', 'nir', 'pixel_qa'])

In [22]:
sr_1

<xarray.Dataset>
Dimensions:   (time: 63, x: 5002, y: 5002)
Coordinates:
  * time      (time) datetime64[ns] 2018-10-04T17:08:24 ... 2019-09-26T17:08:12.480000
  * y         (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x         (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06
Data variables:
    blue      (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    green     (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    red       (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    nir       (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    pixel_qa  (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    ndvi      (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-97.87275...

In [23]:
sr_1 = client.persist(sr_1)

In [24]:
#sr_1 = sr_1.compute(scheduler='threads')

## Mean of NDVI through all year

In [25]:
# Run temporal reductions and rename DataArrays
sr_mean = sr_1.mean('time', keep_attrs=True, skipna=True)

In [26]:
sr_mean

<xarray.Dataset>
Dimensions:   (x: 5002, y: 5002)
Coordinates:
  * y         (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x         (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06
Data variables:
    blue      (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    green     (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    red       (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    nir       (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    pixel_qa  (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    ndvi      (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-97.87275...

In [27]:
sr_mean = sr_mean.rename({'ndvi': 'ndvi_mean'})

In [28]:
sr_mean

<xarray.Dataset>
Dimensions:    (x: 5002, y: 5002)
Coordinates:
  * y          (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x          (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06
Data variables:
    blue       (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    green      (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    red        (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    nir        (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    pixel_qa   (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
    ndvi_mean  (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-97.87275...

In [29]:
sr_mean.ndvi_mean

<xarray.DataArray 'ndvi_mean' (y: 5002, x: 5002)>
dask.array<mean_agg-aggregate, shape=(5002, 5002), dtype=float64, chunksize=(800, 800), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x        (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06
Attributes:
    nodata:   0

In [30]:
sr_mean = sr_mean.apply(to_int)

In [31]:
sr_mean.attrs['crs'] = crs

In [32]:
sr_mean = client.persist(sr_mean)

In [45]:
#sr_mean = sr_mean.compute(scheduler='threads')

In [33]:
nc_filename = '/home/madmex_user/results/s2_2018-2019_39_-30_ndvi_mean_distributed.nc'

In [34]:
#from datacube.drivers.netcdf import write_dataset_to_netcdf

In [36]:
#write_dataset_to_netcdf(sr_mean,nc_filename)

## Interpolation of NDVI values using a defined period

In [37]:
begin=np.datetime_as_string(sr_1['time'].values[0],unit='D')

In [38]:
begin

'2018-10-04'

In [39]:
begin_time = datetime.strptime(begin, '%Y-%m-%d')

In [41]:
sr_1

<xarray.Dataset>
Dimensions:   (time: 63, x: 5002, y: 5002)
Coordinates:
  * time      (time) datetime64[ns] 2018-10-04T17:08:24 ... 2019-09-26T17:08:12.480000
  * y         (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x         (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06
Data variables:
    blue      (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    green     (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    red       (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    nir       (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    pixel_qa  (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
    ndvi      (time, y, x) float64 dask.array<chunksize=(1, 800, 800), meta=np.ndarray>
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...
    geobox:   GeoBox(Geometry({'type': 'Polygon', 'coordinates': [[(-97.87275...

In [43]:
sr_1 = sr_1.chunk({'time': -1, 'x': 800, 'y': 800})

In [52]:
sr_1['time']

<xarray.DataArray 'time' (time: 63)>
array(['2018-10-04T17:08:24.000000000', '2018-10-06T17:05:22.000000000',
       '2018-10-09T17:12:34.000000000', '2018-10-31T17:02:21.000000000',
       '2018-11-05T17:00:45.000000000', '2018-11-08T17:18:03.000000000',
       '2018-11-15T17:08:06.000000000', '2018-11-30T17:08:08.000000000',
       '2018-12-03T17:18:02.000000000', '2018-12-08T17:17:59.000000000',
       '2018-12-15T17:08:03.000000000', '2018-12-18T17:17:59.000000000',
       '2018-12-20T17:08:07.000000000', '2018-12-23T17:18:03.000000000',
       '2019-01-07T17:18:02.000000000', '2019-01-19T17:08:12.000000000',
       '2019-01-22T17:18:07.000000000', '2019-02-03T17:08:09.000000000',
       '2019-02-06T17:18:04.000000000', '2019-02-11T17:18:07.000000000',
       '2019-02-13T17:08:09.000000000', '2019-02-16T17:17:57.000000000',
       '2019-02-16T17:18:04.000000000', '2019-02-26T17:18:03.100000000',
       '2019-02-26T17:18:11.490000000', '2019-03-08T17:18:02.220000000',
       '2019-03-10T17:08:14.720000000', '2019-03-13T17:18:04.950000000',
       '2019-03-28T17:18:07.370000000', '2019-03-30T17:08:14.400000000',
       '2019-04-04T17:08:13.240000000', '2019-04-09T17:08:16.660000000',
       '2019-04-12T17:18:12.640000000', '2019-04-17T17:18:10.450000000',
       '2019-04-19T17:08:18.570000000', '2019-04-22T17:18:14.370000000',
       '2019-04-27T17:18:11.300000000', '2019-05-02T17:18:15.580000000',
       '2019-05-17T17:18:11.450000000', '2019-05-19T17:08:21.090000000',
       '2019-05-22T17:18:16.420000000', '2019-06-06T17:18:09.850000000',
       '2019-06-08T17:08:20.180000000', '2019-07-03T17:08:17.360000000',
       '2019-07-06T17:18:12.820000000', '2019-07-08T17:08:21.250000000',
       '2019-07-11T17:18:16.590000000', '2019-07-31T17:18:15.760000000',
       '2019-08-02T17:08:16.970000000', '2019-08-10T17:18:14.720000000',
       '2019-08-12T17:08:15.990000000', '2019-08-15T17:18:10.900000000',
       '2019-08-17T17:08:18.440000000', '2019-08-22T17:08:14.620000000',
       '2019-08-30T17:18:11.470000000', '2019-09-01T17:08:12.870000000',
       '2019-09-04T17:18:07.560000000', '2019-09-16T17:08:09.730000000',
       '2019-09-16T17:08:16.270000000', '2019-09-19T17:18:06.890000000',
       '2019-09-21T17:08:12.660000000', '2019-09-24T17:18:08.500000000',
       '2019-09-26T17:08:12.480000000'], dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2018-10-04T17:08:24 ... 2019-09-26T17:08:12.480000
Attributes:
    units:    seconds since 1970-01-01 00:00:00

In [44]:
#if using sentinel change to 10D, if using landsat change to 80D
ts_1 = sr_1.ndvi.resample(time='10D', 
                          loffset=begin_time - 
                          datetime.utcfromtimestamp(sr_1['time'][0].values.astype(datetime)*1e-9),
                          keep_attrs=True).nearest().interpolate_na('time')

In [45]:
ts_1

<xarray.DataArray 'ndvi' (time: 36, y: 5002, x: 5002)>
dask.array<transpose, shape=(36, 5002, 5002), dtype=float64, chunksize=(36, 800, 800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2018-10-03T06:51:36 ... 2019-09-18T06:51:36
  * y        (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x        (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06

In [46]:
ts_1['time']

<xarray.DataArray 'time' (time: 36)>
array(['2018-10-03T06:51:36.000000000', '2018-10-13T06:51:36.000000000',
       '2018-10-23T06:51:36.000000000', '2018-11-02T06:51:36.000000000',
       '2018-11-12T06:51:36.000000000', '2018-11-22T06:51:36.000000000',
       '2018-12-02T06:51:36.000000000', '2018-12-12T06:51:36.000000000',
       '2018-12-22T06:51:36.000000000', '2019-01-01T06:51:36.000000000',
       '2019-01-11T06:51:36.000000000', '2019-01-21T06:51:36.000000000',
       '2019-01-31T06:51:36.000000000', '2019-02-10T06:51:36.000000000',
       '2019-02-20T06:51:36.000000000', '2019-03-02T06:51:36.000000000',
       '2019-03-12T06:51:36.000000000', '2019-03-22T06:51:36.000000000',
       '2019-04-01T06:51:36.000000000', '2019-04-11T06:51:36.000000000',
       '2019-04-21T06:51:36.000000000', '2019-05-01T06:51:36.000000000',
       '2019-05-11T06:51:36.000000000', '2019-05-21T06:51:36.000000000',
       '2019-05-31T06:51:36.000000000', '2019-06-10T06:51:36.000000000',
       '2019-06-20T06:51:36.000000000', '2019-06-30T06:51:36.000000000',
       '2019-07-10T06:51:36.000000000', '2019-07-20T06:51:36.000000000',
       '2019-07-30T06:51:36.000000000', '2019-08-09T06:51:36.000000000',
       '2019-08-19T06:51:36.000000000', '2019-08-29T06:51:36.000000000',
       '2019-09-08T06:51:36.000000000', '2019-09-18T06:51:36.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2018-10-03T06:51:36 ... 2019-09-18T06:51:36
Attributes:
    units:    seconds since 1970-01-01 00:00:00

In [47]:
times=np.datetime_as_string(ts_1['time'],unit='D')

In [48]:
times

array(['2018-10-03', '2018-10-13', '2018-10-23', '2018-11-02',
       '2018-11-12', '2018-11-22', '2018-12-02', '2018-12-12',
       '2018-12-22', '2019-01-01', '2019-01-11', '2019-01-21',
       '2019-01-31', '2019-02-10', '2019-02-20', '2019-03-02',
       '2019-03-12', '2019-03-22', '2019-04-01', '2019-04-11',
       '2019-04-21', '2019-05-01', '2019-05-11', '2019-05-21',
       '2019-05-31', '2019-06-10', '2019-06-20', '2019-06-30',
       '2019-07-10', '2019-07-20', '2019-07-30', '2019-08-09',
       '2019-08-19', '2019-08-29', '2019-09-08', '2019-09-18'],
      dtype='<U28')

In [49]:
ts_1_times_clean = ts_1.assign_coords({'time':times})

In [50]:
ts_1_times_clean

<xarray.DataArray 'ndvi' (time: 36, y: 5002, x: 5002)>
dask.array<transpose, shape=(36, 5002, 5002), dtype=float64, chunksize=(36, 800, 800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) <U28 '2018-10-03' '2018-10-13' ... '2019-09-08' '2019-09-18'
  * y        (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x        (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06

In [51]:
ts_1_times_clean = client.persist(ts_1_times_clean)

# Segmentation using NDVI mean 

In [53]:
from django.contrib.gis.geos.geometry import GEOSGeometry
import gc
from importlib import import_module
import os

from madmex.settings import TEMP_DIR
from madmex.util import parser_extra_args

In [54]:
algorithm = 'bis' #'slic'
name = 'NDVI_mean_03_12_2019_sentinel2'
extra_args = parser_extra_args(['t=40','s=0.5','c=0.7']) #for slic: parser_extra_args(['n_segments=10000', 'compactness=0.5'])

In [55]:
band_list = ['ndvi_mean']

In [56]:
module = import_module('madmex.segmentation.%s' % algorithm)
Segmentation = module.Segmentation

In [57]:
path = os.path.join(TEMP_DIR) + '/segmentation_results/'

In [58]:
path

'/shared_volume/temp/segmentation_results/'

In [59]:
if not os.path.exists(path):
    os.makedirs(path)

In [60]:
seg = Segmentation.from_geoarray(sr_mean, **extra_args)

In [61]:
seg.geobox.extent.wkt

'POLYGON ((2927940 976140,2927940 926120,2977960 926120,2977960 976140,2927940 976140))'

In [62]:
seg.segment()


In [63]:
seg.segments_array

array([[165086, 165086, 165781, ..., 165780, 165780, 165780],
       [165086, 165781, 165781, ..., 165780, 165780, 165780],
       [165086, 165781, 165781, ..., 165780, 165780, 165780],
       ...,
       [     0,      0,      0, ...,    439,    439,    439],
       [     0,      0,      0, ...,    439,    439,    439],
       [     0,      0,      0, ...,    439,    439,    439]], dtype=int32)

In [64]:
fc = seg.polygonize()

In [65]:
fc[0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[2929200.0, 976140.0],
    [2929270.0, 976140.0],
    [2929270.0, 976110.0],
    [2929260.0, 976110.0],
    [2929230.0, 976110.0],
    [2929230.0, 976130.0],
    [2929210.0, 976130.0],
    [2929210.0, 976120.0],
    [2929200.0, 976120.0],
    [2929200.0, 976140.0]]]},
 'properties': {'id': 166182.0}}

In [66]:
name_file = name + '_' + algorithm + '_t_' + str(extra_args['t']) +'_s_'+ str(extra_args['s']) +'_c_'+ str(extra_args['c']) + '.shp'

In [67]:
name_file

'NDVI_mean_03_12_2019_sentinel2_bis_t_40_s_0.5_c_0.7.shp'

In [299]:
shp_path = seg.to_shapefile(filename=path + '/' +name_file, fc=fc)


In [300]:
shp_path

'/shared_volume/temp/segmentation_results//NDVI_mean_03_12_2019_sentinel2_bis_t_40_s_0.5_c_0.7.shp'

## Unsupervised clustering based on a date of ts_1_times_clean

The idea is label segments (generated from segmentation based in ndvi_mean) with a value based on unsupervised clustering (K Means) using a date of DataArray ts_1_times_clean (for example 3rd date)

For each segment we need to calculate a mean of the ndvi value (interpolated) that surrounds a number of pixels for a given date. We use already implemented zonal statistics in antares3

https://github.com/CONABIO/antares3/blob/rapideye-support/madmex/overlay/extractions.py

In [149]:
from operator import itemgetter

from affine import Affine
from rasterio.features import rasterize
from collections import OrderedDict


from madmex.overlay.conversions import rasterize_xarray
from madmex.util import chunk

In [150]:
fc[0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[2929200.0, 976140.0],
    [2929270.0, 976140.0],
    [2929270.0, 976110.0],
    [2929260.0, 976110.0],
    [2929230.0, 976110.0],
    [2929230.0, 976130.0],
    [2929210.0, 976130.0],
    [2929210.0, 976120.0],
    [2929200.0, 976120.0],
    [2929200.0, 976140.0]]]},
 'properties': {'id': 166182.0}}

In [152]:
fc_subset=[(x['properties']['id'], x['geometry']) for x in fc]

In [153]:
fc_subset[0]

(166182.0,
 {'type': 'Polygon',
  'coordinates': [[[2929200.0, 976140.0],
    [2929270.0, 976140.0],
    [2929270.0, 976110.0],
    [2929260.0, 976110.0],
    [2929230.0, 976110.0],
    [2929230.0, 976130.0],
    [2929210.0, 976130.0],
    [2929210.0, 976120.0],
    [2929200.0, 976120.0],
    [2929200.0, 976140.0]]]})

In [154]:
fc_sorted = sorted(fc_subset, key=itemgetter(0))

In [155]:
fc_sorted[0]

(0.0,
 {'type': 'Polygon',
  'coordinates': [[[2928000.0, 926280.0],
    [2928010.0, 926280.0],
    [2928010.0, 926270.0],
    [2928030.0, 926270.0],
    [2928030.0, 926260.0],
    [2928040.0, 926260.0],
    [2928040.0, 926250.0],
    [2928050.0, 926250.0],
    [2928050.0, 926230.0],
    [2928090.0, 926230.0],
    [2928090.0, 926220.0],
    [2928100.0, 926220.0],
    [2928100.0, 926170.0],
    [2928090.0, 926170.0],
    [2928090.0, 926150.0],
    [2928080.0, 926150.0],
    [2928080.0, 926130.0],
    [2928070.0, 926130.0],
    [2928070.0, 926120.0],
    [2927940.0, 926120.0],
    [2927940.0, 926260.0],
    [2927970.0, 926260.0],
    [2927980.0, 926260.0],
    [2927980.0, 926270.0],
    [2927990.0, 926270.0],
    [2928000.0, 926270.0],
    [2928000.0, 926280.0]],
   [[2928060.0, 926180.0],
    [2928070.0, 926180.0],
    [2928070.0, 926210.0],
    [2928060.0, 926210.0],
    [2928060.0, 926220.0],
    [2928030.0, 926220.0],
    [2928030.0, 926170.0],
    [2928060.0, 926170.0],
    [2928060

In [157]:
sr_0.affine

Affine(10.0, 0.0, 2927940.0,
       0.0, -10.0, 976140.0)

In [72]:
list(sr_0.affine)

[10.0, 0.0, 2927940.0, 0.0, -10.0, 976140.0, 0.0, 0.0, 1.0]

In [78]:
list(sr_0.affine)[0:6]

[10.0, 0.0, 2927940.0, 0.0, -10.0, 976140.0]

In [79]:
aff = Affine(*list(sr_0.affine)[0:6])

In [80]:
dimensions_dataset = list(sr_0.coords)
list_dimensions = [x for x in dimensions_dataset if x != 'time']
lambda_function = lambda l_netcdf,l_test: l_netcdf[0] if l_netcdf[0] in l_test else l_netcdf[1]
xdim = lambda_function(list_dimensions,['x','longitude'])
ydim = lambda_function(list_dimensions,['y','latitude'])

In [81]:
xdim,ydim

('x', 'y')

In [160]:
len([k[0] for k in fc_sorted])

166193

First position of fc_sorted is id, second position is geometry

In [161]:
iterable_zip = zip([k[1] for k in fc_sorted], [k[0] for k in fc_sorted])

In [162]:
iterable_zip

**fc_raster is seg.segments_array !!!**

In [163]:
fc_raster = rasterize(iterable_zip, transform=aff,
                      out_shape=(sr_0.sizes[ydim], sr_0.sizes[xdim]),
                      dtype='float64', fill=np.nan)

In [164]:
fc_raster

array([[165086., 165086., 165781., ..., 165780., 165780., 165780.],
       [165086., 165781., 165781., ..., 165780., 165780., 165780.],
       [165086., 165781., 165781., ..., 165780., 165780., 165780.],
       ...,
       [     0.,      0.,      0., ...,    439.,    439.,    439.],
       [     0.,      0.,      0., ...,    439.,    439.,    439.],
       [     0.,      0.,      0., ...,    439.,    439.,    439.]])

In [165]:
seg.segments_array

array([[165086, 165086, 165781, ..., 165780, 165780, 165780],
       [165086, 165781, 165781, ..., 165780, 165780, 165780],
       [165086, 165781, 165781, ..., 165780, 165780, 165780],
       ...,
       [     0,      0,      0, ...,    439,    439,    439],
       [     0,      0,      0, ...,    439,    439,    439],
       [     0,      0,      0, ...,    439,    439,    439]], dtype=int32)

In [166]:
fc_raster = seg.segments_array

In [167]:
fc_raster.shape

(5002, 5002)

In [168]:
fc_dataarray = xr.DataArray(fc_raster, dims=[ydim, xdim], name='features_id')

In [169]:
fc_dataarray

<xarray.DataArray 'features_id' (y: 5002, x: 5002)>
array([[165086, 165086, 165781, ..., 165780, 165780, 165780],
       [165086, 165781, 165781, ..., 165780, 165780, 165780],
       [165086, 165781, 165781, ..., 165780, 165780, 165780],
       ...,
       [     0,      0,      0, ...,    439,    439,    439],
       [     0,      0,      0, ...,    439,    439,    439],
       [     0,      0,      0, ...,    439,    439,    439]], dtype=int32)
Dimensions without coordinates: y, x

In [170]:
ts_1_times_clean

<xarray.DataArray 'ndvi' (time: 36, y: 5002, x: 5002)>
dask.array<transpose, shape=(36, 5002, 5002), dtype=float64, chunksize=(36, 800, 800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) <U28 '2018-10-03' '2018-10-13' ... '2019-09-08' '2019-09-18'
  * y        (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x        (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06

In [171]:
ts_1_times_clean['time']

<xarray.DataArray 'time' (time: 36)>
array(['2018-10-03', '2018-10-13', '2018-10-23', '2018-11-02', '2018-11-12',
       '2018-11-22', '2018-12-02', '2018-12-12', '2018-12-22', '2019-01-01',
       '2019-01-11', '2019-01-21', '2019-01-31', '2019-02-10', '2019-02-20',
       '2019-03-02', '2019-03-12', '2019-03-22', '2019-04-01', '2019-04-11',
       '2019-04-21', '2019-05-01', '2019-05-11', '2019-05-21', '2019-05-31',
       '2019-06-10', '2019-06-20', '2019-06-30', '2019-07-10', '2019-07-20',
       '2019-07-30', '2019-08-09', '2019-08-19', '2019-08-29', '2019-09-08',
       '2019-09-18'], dtype='<U28')
Coordinates:
  * time     (time) <U28 '2018-10-03' '2018-10-13' ... '2019-09-08' '2019-09-18'

In [172]:
date_of_ndvi = 2
ts_1_times_clean.isel(time=date_of_ndvi)

<xarray.DataArray 'ndvi' (y: 5002, x: 5002)>
dask.array<getitem, shape=(5002, 5002), dtype=float64, chunksize=(800, 800), chunktype=numpy.ndarray>
Coordinates:
    time     <U28 '2018-10-23'
  * y        (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x        (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06

In [173]:
combined = xr.merge([fc_dataarray, ts_1_times_clean.isel(time=date_of_ndvi)])

In [174]:
combined

<xarray.Dataset>
Dimensions:      (x: 5002, y: 5002)
Coordinates:
  * x            (x) float64 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06
  * y            (y) float64 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
    time         <U28 '2018-10-23'
Data variables:
    features_id  (y, x) int32 165086 165086 165781 165781 ... 439 439 439 439
    ndvi         (y, x) float64 dask.array<chunksize=(800, 800), meta=np.ndarray>

In [175]:
combined = combined.stack(z=(xdim, ydim))

In [176]:
combined

<xarray.Dataset>
Dimensions:      (z: 25020004)
Coordinates:
    time         <U28 '2018-10-23'
  * z            (z) MultiIndex
  - x            (z) float64 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06
  - y            (z) float64 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
Data variables:
    features_id  (z) int32 165086 165086 165086 165086 ... 439 439 439 439
    ndvi         (z) float64 dask.array<chunksize=(570228,), meta=np.ndarray>

In [177]:
combined=combined.reset_index('z')

In [178]:
combined

<xarray.Dataset>
Dimensions:      (z: 25020004)
Coordinates:
    time         <U28 '2018-10-23'
    x            (z) float64 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06
    y            (z) float64 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
Dimensions without coordinates: z
Data variables:
    features_id  (z) int32 165086 165086 165086 165086 ... 439 439 439 439
    ndvi         (z) float64 dask.array<chunksize=(570228,), meta=np.ndarray>

In [179]:
combined=combined.drop([xdim,ydim])

In [180]:
combined

<xarray.Dataset>
Dimensions:      (z: 25020004)
Coordinates:
    time         <U28 '2018-10-23'
Dimensions without coordinates: z
Data variables:
    features_id  (z) int32 165086 165086 165086 165086 ... 439 439 439 439
    ndvi         (z) float64 dask.array<chunksize=(570228,), meta=np.ndarray>

In [181]:
combined = combined.where(np.isfinite(combined['features_id']), drop=True)

In [182]:
combined

<xarray.Dataset>
Dimensions:      (z: 25020004)
Coordinates:
    time         <U28 '2018-10-23'
Dimensions without coordinates: z
Data variables:
    features_id  (z) float64 1.651e+05 1.651e+05 1.651e+05 ... 439.0 439.0 439.0
    ndvi         (z) float64 dask.array<chunksize=(570228,), meta=np.ndarray>

In [183]:
df = combined.to_dataframe()

In [184]:
df

,features_id,ndvi,time
z,,,
0,165086.0,8490.737089,2018-10-23
1,165086.0,8451.722790,2018-10-23
2,165086.0,8448.037870,2018-10-23
3,165086.0,8459.241447,2018-10-23
4,165086.0,8497.457186,2018-10-23
...,...,...,...
25019999,439.0,7855.871886,2018-10-23
25020000,439.0,7787.790043,2018-10-23
25020001,439.0,7711.171662,2018-10-23


In [185]:
df.drop(['time'], axis=1, inplace=True)

In [186]:
df

,features_id,ndvi
z,,
0,165086.0,8490.737089
1,165086.0,8451.722790
2,165086.0,8448.037870
3,165086.0,8459.241447
4,165086.0,8497.457186
...,...,...
25019999,439.0,7855.871886
25020000,439.0,7787.790043
25020001,439.0,7711.171662


In [187]:
ts_1_times_clean

<xarray.DataArray 'ndvi' (time: 36, y: 5002, x: 5002)>
dask.array<transpose, shape=(36, 5002, 5002), dtype=float64, chunksize=(36, 800, 800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) <U28 '2018-10-03' '2018-10-13' ... '2019-09-08' '2019-09-18'
  * y        (y) float64 9.761e+05 9.761e+05 9.761e+05 ... 9.261e+05 9.261e+05
  * x        (x) float64 2.928e+06 2.928e+06 2.928e+06 ... 2.978e+06 2.978e+06

In [188]:
ts_1_times_clean.name

'ndvi'

In [189]:
agg_list = [(ts_1_times_clean.name , 'mean')]


In [190]:
agg_list

[('ndvi', 'mean')]

In [191]:
agg_ordered_dict = OrderedDict(agg_list)

In [192]:
agg_ordered_dict

OrderedDict([('ndvi', 'mean')])

In [193]:
df = df.groupby('features_id').agg(agg_ordered_dict)

In [194]:
df

,ndvi
features_id,
0.0,8056.663160
1.0,8637.538547
2.0,8607.380368
3.0,7347.080730
4.0,8615.343932
...,...
166188.0,3975.137917
166189.0,7965.173755
166190.0,5529.615919


In [200]:
df = df.reset_index('features_id')

In [201]:
df

,features_id,ndvi
0,0.0,8056.663160
1,1.0,8637.538547
2,2.0,8607.380368
3,3.0,7347.080730
4,4.0,8615.343932
...,...,...
166188,166188.0,3975.137917
166189,166189.0,7965.173755
166190,166190.0,5529.615919
166191,166191.0,2718.212998


**Join df with fc in geopandas dataframe form**

In [195]:
import geopandas as gpd

In [197]:
fc[0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[2929200.0, 976140.0],
    [2929270.0, 976140.0],
    [2929270.0, 976110.0],
    [2929260.0, 976110.0],
    [2929230.0, 976110.0],
    [2929230.0, 976130.0],
    [2929210.0, 976130.0],
    [2929210.0, 976120.0],
    [2929200.0, 976120.0],
    [2929200.0, 976140.0]]]},
 'properties': {'id': 166182.0}}

In [263]:
gdf = gpd.GeoDataFrame.from_features([feature for feature in fc], crs = var[1].geobox.crs.crs_str)

In [264]:
gdf.head()

,geometry,id
0,"POLYGON ((2929200.000 976140.000, 2929270.000 ...",166182.0
1,"POLYGON ((2930600.000 976140.000, 2930620.000 ...",166171.0
2,"POLYGON ((2932180.000 976140.000, 2932280.000 ...",166153.0
3,"POLYGON ((2933070.000 976140.000, 2933110.000 ...",166155.0
4,"POLYGON ((2933850.000 976140.000, 2933950.000 ...",166156.0


In [265]:
gdf = gdf.rename(columns={'id':'features_id'})

In [266]:
gdf

,geometry,features_id
0,"POLYGON ((2929200.000 976140.000, 2929270.000 ...",166182.0
1,"POLYGON ((2930600.000 976140.000, 2930620.000 ...",166171.0
2,"POLYGON ((2932180.000 976140.000, 2932280.000 ...",166153.0
3,"POLYGON ((2933070.000 976140.000, 2933110.000 ...",166155.0
4,"POLYGON ((2933850.000 976140.000, 2933950.000 ...",166156.0
...,...,...
166188,"POLYGON ((2976280.000 926230.000, 2976320.000 ...",425.0
166189,"POLYGON ((2976320.000 926300.000, 2976340.000 ...",426.0
166190,"POLYGON ((2976600.000 926160.000, 2976630.000 ...",430.0
166191,"POLYGON ((2977400.000 926250.000, 2977450.000 ...",436.0


In [267]:
gdf_df_join = gdf.merge(df, on='features_id', how='left')

In [268]:
gdf_df_join

,geometry,features_id,ndvi
0,"POLYGON ((2929200.000 976140.000, 2929270.000 ...",166182.0,5640.791542
1,"POLYGON ((2930600.000 976140.000, 2930620.000 ...",166171.0,6773.996255
2,"POLYGON ((2932180.000 976140.000, 2932280.000 ...",166153.0,3987.455400
3,"POLYGON ((2933070.000 976140.000, 2933110.000 ...",166155.0,6115.517861
4,"POLYGON ((2933850.000 976140.000, 2933950.000 ...",166156.0,7506.138528
...,...,...,...
166188,"POLYGON ((2976280.000 926230.000, 2976320.000 ...",425.0,7785.746269
166189,"POLYGON ((2976320.000 926300.000, 2976340.000 ...",426.0,8023.923929
166190,"POLYGON ((2976600.000 926160.000, 2976630.000 ...",430.0,6923.699275
166191,"POLYGON ((2977400.000 926250.000, 2977450.000 ...",436.0,7753.087556


In [302]:
gdf_df_join[gdf_df_join.features_id==166192]

,geometry,features_id,ndvi,preds
31,"POLYGON ((2974130.000 976140.000, 2974230.000 ...",166192.0,2083.206478,4


In [269]:
path

'/shared_volume/temp/segmentation_results/'

In [270]:
name_file

'NDVI_mean_03_12_2019_sentinel2_bis_t_40_s_0.5_c_0.7.shp'

In [271]:
name_file_gpd = name_file.split('.shp')[0] + '_gpd' + '.shp'

In [272]:
name_file_gpd

'NDVI_mean_03_12_2019_sentinel2_bis_t_40_s_0.5_c_0.7_gpd.shp'

In [273]:
gdf_df_features = gdf_df_join.drop(['ndvi'],axis=1)

In [274]:
gdf_df_features

,geometry,features_id
0,"POLYGON ((2929200.000 976140.000, 2929270.000 ...",166182.0
1,"POLYGON ((2930600.000 976140.000, 2930620.000 ...",166171.0
2,"POLYGON ((2932180.000 976140.000, 2932280.000 ...",166153.0
3,"POLYGON ((2933070.000 976140.000, 2933110.000 ...",166155.0
4,"POLYGON ((2933850.000 976140.000, 2933950.000 ...",166156.0
...,...,...
166188,"POLYGON ((2976280.000 926230.000, 2976320.000 ...",425.0
166189,"POLYGON ((2976320.000 926300.000, 2976340.000 ...",426.0
166190,"POLYGON ((2976600.000 926160.000, 2976630.000 ...",430.0
166191,"POLYGON ((2977400.000 926250.000, 2977450.000 ...",436.0


In [297]:
path + name_file_gpd

'/shared_volume/temp/segmentation_results/NDVI_mean_03_12_2019_sentinel2_bis_t_40_s_0.5_c_0.7_gpd.shp'

In [298]:
gdf_df_features.to_file(path + name_file_gpd)

## Unsupervised clustering

In [223]:
from sklearn.cluster import KMeans


In [275]:
gdf_df_join[['ndvi']]

,ndvi
0,5640.791542
1,6773.996255
2,3987.455400
3,6115.517861
4,7506.138528
...,...
166188,7785.746269
166189,8023.923929
166190,6923.699275
166191,7753.087556


In [276]:
kmeans = KMeans(n_clusters=10)


In [277]:
kmeans.fit(gdf_df_join[['ndvi']].fillna(0).values)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [278]:
preds = kmeans.predict(gdf_df_join[['ndvi']].fillna(0).values)



In [279]:
preds

array([7, 8, 6, ..., 8, 2, 8], dtype=int32)

In [280]:
preds.shape

(166193,)

In [281]:
# Get centers of the clusters
centers = kmeans.cluster_centers_

In [282]:
centers

array([[6169.07543129],
       [3040.25643342],
       [7560.50018749],
       [4725.10115779],
       [1885.50644302],
       [8298.34246994],
       [3926.44194343],
       [5462.26309755],
       [6861.55329602],
       [ 118.06381337]])

In [283]:
centers.shape

(10, 1)

In [ ]:
gdf_df_join.plot(column='ndvi',c=preds)

In [284]:
gdf_df_join['preds'] = preds

In [285]:
gdf_df_join

,geometry,features_id,ndvi,preds
0,"POLYGON ((2929200.000 976140.000, 2929270.000 ...",166182.0,5640.791542,7
1,"POLYGON ((2930600.000 976140.000, 2930620.000 ...",166171.0,6773.996255,8
2,"POLYGON ((2932180.000 976140.000, 2932280.000 ...",166153.0,3987.455400,6
3,"POLYGON ((2933070.000 976140.000, 2933110.000 ...",166155.0,6115.517861,0
4,"POLYGON ((2933850.000 976140.000, 2933950.000 ...",166156.0,7506.138528,2
...,...,...,...,...
166188,"POLYGON ((2976280.000 926230.000, 2976320.000 ...",425.0,7785.746269,2
166189,"POLYGON ((2976320.000 926300.000, 2976340.000 ...",426.0,8023.923929,5
166190,"POLYGON ((2976600.000 926160.000, 2976630.000 ...",430.0,6923.699275,8
166191,"POLYGON ((2977400.000 926250.000, 2977450.000 ...",436.0,7753.087556,2


In [286]:
name_file_ndvi = name_file.split('.shp')[0] + '_ndvi' + '.shp'

In [287]:
gdf_df_join_ndvi = gdf_df_join.drop(['features_id','preds'],axis=1)

In [288]:
gdf_df_join_ndvi

,geometry,ndvi
0,"POLYGON ((2929200.000 976140.000, 2929270.000 ...",5640.791542
1,"POLYGON ((2930600.000 976140.000, 2930620.000 ...",6773.996255
2,"POLYGON ((2932180.000 976140.000, 2932280.000 ...",3987.455400
3,"POLYGON ((2933070.000 976140.000, 2933110.000 ...",6115.517861
4,"POLYGON ((2933850.000 976140.000, 2933950.000 ...",7506.138528
...,...,...
166188,"POLYGON ((2976280.000 926230.000, 2976320.000 ...",7785.746269
166189,"POLYGON ((2976320.000 926300.000, 2976340.000 ...",8023.923929
166190,"POLYGON ((2976600.000 926160.000, 2976630.000 ...",6923.699275
166191,"POLYGON ((2977400.000 926250.000, 2977450.000 ...",7753.087556


In [303]:
gdf_df_join_ndvi[gdf_df_join.features_id==166192]

,geometry,ndvi
31,"POLYGON ((2974130.000 976140.000, 2974230.000 ...",2083.206478


In [295]:
path + name_file_ndvi

'/shared_volume/temp/segmentation_results/NDVI_mean_03_12_2019_sentinel2_bis_t_40_s_0.5_c_0.7_ndvi.shp'

In [296]:
gdf_df_join_ndvi.to_file(path + name_file_ndvi)

In [289]:
name_file_cluster = name_file.split('.shp')[0] + '_cluster' + '.shp'

In [290]:
name_file_cluster

'NDVI_mean_03_12_2019_sentinel2_bis_t_40_s_0.5_c_0.7_cluster.shp'

In [291]:
gdf_df_cluster = gdf_df_join.drop(['features_id','ndvi'],axis=1)

In [292]:
gdf_df_cluster

,geometry,preds
0,"POLYGON ((2929200.000 976140.000, 2929270.000 ...",7
1,"POLYGON ((2930600.000 976140.000, 2930620.000 ...",8
2,"POLYGON ((2932180.000 976140.000, 2932280.000 ...",6
3,"POLYGON ((2933070.000 976140.000, 2933110.000 ...",0
4,"POLYGON ((2933850.000 976140.000, 2933950.000 ...",2
...,...,...
166188,"POLYGON ((2976280.000 926230.000, 2976320.000 ...",2
166189,"POLYGON ((2976320.000 926300.000, 2976340.000 ...",5
166190,"POLYGON ((2976600.000 926160.000, 2976630.000 ...",8
166191,"POLYGON ((2977400.000 926250.000, 2977450.000 ...",2


In [304]:
gdf_df_cluster[gdf_df_join.features_id==166192]

,geometry,preds
31,"POLYGON ((2974130.000 976140.000, 2974230.000 ...",4


In [293]:
gdf_df_cluster.to_file(path + name_file_cluster)

In [294]:
path + name_file_cluster

'/shared_volume/temp/segmentation_results/NDVI_mean_03_12_2019_sentinel2_bis_t_40_s_0.5_c_0.7_cluster.shp'